<a href="https://colab.research.google.com/github/KenjiMatsumoto/ChainerHandOn/blob/master/audio_colabolatory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
sys.path.append('..')
import random
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn import model_selection
from sklearn import preprocessing
import IPython.display as ipd
import keras
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation
from keras.layers import Conv2D, GlobalAveragePooling2D
from keras.layers import BatchNormalization, Add
from keras.layers import Flatten, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.models import Sequential
import cv2
from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# define directories
base_dir = "drive/My Drive/"
train_dir = os.path.join(base_dir, "train")
class_train_file = os.path.join(base_dir, "class_train.tsv")
# submit_file = os.path.join(base_dir, "sample_submit.tsv")

In [3]:
# load metadata
class_train_data = pd.read_table(class_train_file, header=None, names=('filename', 'category'))
# 性別の列を追加
class_train_data['sex'] = 0
# 年齢層の列を追加
class_train_data['ageType'] = 0
class_train_data

,filename,category,sex,ageType
0,0002f1cd968ca78ada9e1c7037224773,MA_CH,0,0
1,0003747ec9268461d4cbb9e1b86e9663,FE_AD,0,0
2,0003b32f378b001f0f73bf0981da8773,MA_CH,0,0
3,0004ab975bf8b59e1b19f2b7b6d1548b,MA_CH,0,0
4,0005678b57ca265a65f8ef0cc7481277,MA_AD,0,0
5,00068ee0420a8dac9f7b51b1783b7f4b,FE_EL,0,0
6,0007a9416dd92cc01275be7e3aac6f9f,FE_CH,0,0
7,000841455294bab27f3ecc5e8829f2df,MA_AD,0,0
8,000ad3f1ea1fb1eafa92d9fa818c5122,MA_AD,0,0
9,000b1bbde96b2c541ee1721dfdda6d7a,FE_CH,0,0


In [4]:
# get data size
data_size = class_train_data.shape
print(data_size)

# arrange target label and its name
for i in range(data_size[0]):
    if 'FE_' in class_train_data.iloc[i,1]:
        # 女性の場合は、1を設定。男性はデフォルトで0を設定。
        class_train_data.iloc[i,2] = 1
    if '_CH' in class_train_data.iloc[i,1]:
        # 子供の場合、0を設定
        class_train_data.iloc[i,3] = 0
    elif '_AD' in class_train_data.iloc[i,1]:
        # 大人の場合、1を設定
        class_train_data.iloc[i,3] = 1
    else:
        # 高齢者の場合、2を設定
        class_train_data.iloc[i,3] = 2
class_train_data

(37806, 4)


,filename,category,sex,ageType
0,0002f1cd968ca78ada9e1c7037224773,MA_CH,0,0
1,0003747ec9268461d4cbb9e1b86e9663,FE_AD,1,1
2,0003b32f378b001f0f73bf0981da8773,MA_CH,0,0
3,0004ab975bf8b59e1b19f2b7b6d1548b,MA_CH,0,0
4,0005678b57ca265a65f8ef0cc7481277,MA_AD,0,1
5,00068ee0420a8dac9f7b51b1783b7f4b,FE_EL,1,2
6,0007a9416dd92cc01275be7e3aac6f9f,FE_CH,1,0
7,000841455294bab27f3ecc5e8829f2df,MA_AD,0,1
8,000ad3f1ea1fb1eafa92d9fa818c5122,MA_AD,0,1
9,000b1bbde96b2c541ee1721dfdda6d7a,FE_CH,1,0


In [5]:
# get training dataset and target dataset
x = list(class_train_data.loc[:,"filename"] + '.wav')
y = list(class_train_data.loc[:, "sex"])

x_train, x_validation, y_train, y_validation = model_selection.train_test_split(x, y, test_size=0.25, stratify=y)
print("x train:{0}\ny train:{1}\nx validation:{2}\ny validation:{3}".format(len(x_train),
                                                                len(y_train), 
                                                                len(x_validation), 
                                                                len(y_validation)))

x train:28354
y train:28354
x validation:9452
y validation:9452


In [0]:
freq = 128
time = 1723
# save wave data in npz, with augmentation
def save_np_data(filename, x, y, y_start_index, y_end_index, x_data_size, aug=None, rates=None):
    np_data = np.zeros(freq*time*x_data_size).reshape(x_data_size, freq, time)
    np_targets = np.zeros(y_end_index - y_start_index)
    index = 0
    for i in range(y_start_index, y_end_index):
        _x, fs = Common.load_wave_data(train_dir, x[i])
        if aug is not None:
            _x = aug(x=_x, rate=rates[i])
        _x = Common.calculate_melsp(_x)
        # resizeで足りない秒数は0埋めにする
        res = cv2.resize(_x, dsize=(time, freq), interpolation=cv2.INTER_CUBIC)
        np_data[index] = res
        np_targets[index] = y[i]
        index += 1
    np.savez(filename, x=np_data, y=np_targets)

# # save test dataset
# if not os.path.exists("esc_melsp_test.npz"):
# save_np_data("esc_melsp_test1.npz", x_validation,  y_validation, 0, 1180, 1180)
# save_np_data("esc_melsp_test2.npz", x_validation,  y_validation, 1601, 2360, 1180)
# save_np_data("esc_melsp_test3.npz", x_validation,  y_validation, 2361, 3540, 1180)
# save_np_data("esc_melsp_test4.npz", x_validation,  y_validation, 3541, 4720, 1180)
# save_np_data("esc_melsp_test5.npz", x_validation,  y_validation, 4721, 5900, 1180)
# save_np_data("esc_melsp_test6.npz", x_validation,  y_validation, 5901, 7080, 1180)
# save_np_data("esc_melsp_test7.npz", x_validation,  y_validation, 7081, 8260, 1180)
# save_np_data("esc_melsp_test8.npz", x_validation,  y_validation, 8261, 9452, 1191)

# # save raw training dataset
# save_np_data("esc_melsp_train_raw1.npz", x_train,  y_train, 0, 3540, 3540)
# save_np_data("esc_melsp_train_raw2.npz", x_train,  y_train, 3541, 7080, 3540)
# save_np_data("esc_melsp_train_raw3.npz", x_train,  y_train, 7081, 10620, 3540)
# save_np_data("esc_melsp_train_raw4.npz", x_train,  y_train, 10621, 14160, 3540)
# save_np_data("esc_melsp_train_raw5.npz", x_train,  y_train, 14161, 17700, 3540)
# save_np_data("esc_melsp_train_raw6.npz", x_train,  y_train, 17701, 21240, 3540)
# save_np_data("esc_melsp_train_raw7.npz", x_train,  y_train, 21241, 24780, 3540)
# save_np_data("esc_melsp_train_raw8.npz", x_train,  y_train, 24781, 28354, 3573)

# # save training dataset with white noise
# if not os.path.exists("esc_melsp_train_wn.npz"):
#     rates = np.random.randint(1,50,len(x_train))/10000
#     save_np_data("esc_melsp_train_wn.npz", x_train,  y_train, aug=add_white_noise, rates=rates)

# # save training dataset with sound shift
# if not os.path.exists("esc_melsp_train_ss.npz"):
#     rates = np.random.choice(np.arange(2,6),len(y_train))
#     save_np_data("esc_melsp_train_ss.npz", x_train,  y_train, aug=shift_sound, rates=rates)

# # save training dataset with stretch
# if not os.path.exists("esc_melsp_train_st.npz"):
#     rates = np.random.choice(np.arange(80,120),len(y_train))/100
#     save_np_data("esc_melsp_train_st.npz", x_train,  y_train, aug=stretch_sound, rates=rates)

# # save training dataset with combination of white noise and shift or stretch
# if not os.path.exists("esc_melsp_train_com.npz"):
#     np_data = np.zeros(freq*time*len(x_train)).reshape(len(x_train), freq, time)
#     np_targets = np.zeros(len(y_train))
#     for i in range(len(y_train)):
#         x, fs = load_wave_data(train_dir, x_train[i])
#         x = add_white_noise(x=x, rate=np.random.randint(1,50)/1000)
#         if np.random.choice((True,False)):
#             x = shift_sound(x=x, rate=np.random.choice(np.arange(2,6)))
#         else:
#             x = stretch_sound(x=x, rate=np.random.choice(np.arange(80,120))/100)
#         x = calculate_melsp(x)
#         np_data[i] = x
#         np_targets[i] = y_train[i]
#     np.savez("esc_melsp_train_com.npz", x=np_data, y=np_targets)

In [0]:
#trainデータ読み込み
archive = np.load("drive/My Drive/esc_melsp_train_raw1.npz")
archive.files
x_train = archive["x"]
y_train = archive["y"]

# testデータ読み込み
archive = np.load("drive/My Drive/esc_melsp_test1.npz")
archive.files
x_test = archive["x"]
y_test = archive["y"]

In [8]:
print("x train:{0}\ny train:{1}\nx test:{2}\ny test:{3}".format(x_train.shape,
                                                                y_train.shape, 
                                                                x_test.shape, 
                                                                y_test.shape))

x train:(3540, 128, 1723)
y train:(3540,)
x test:(1180, 128, 1723)
y test:(1180,)


In [9]:
train_num = 3540
test_num = 1180
# redefine target data into one hot vector
classes = 2
y_train = keras.utils.to_categorical(y_train, classes)
y_test = keras.utils.to_categorical(y_test, classes)

# reshape training dataset
x_train = x_train.reshape(train_num, freq, time, 1)
x_test = x_test.reshape(test_num, freq, time, 1)

print("x train:{0}\ny train:{1}\nx test:{2}\ny test:{3}".format(x_train.shape,
                                                                y_train.shape, 
                                                                x_test.shape, 
                                                                y_test.shape))

x train:(3540, 128, 1723, 1)
y train:(3540, 2)
x test:(1180, 128, 1723, 1)
y test:(1180, 2)


In [10]:
def cba(inputs, filters, kernel_size, strides):
    x = Conv2D(filters, kernel_size=kernel_size, strides=strides, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

# define CNN
inputs = Input(shape=(x_train.shape[1:]))

x_1 = cba(inputs, filters=32, kernel_size=(1,8), strides=(1,2))
x_1 = cba(x_1, filters=32, kernel_size=(8,1), strides=(2,1))
x_1 = cba(x_1, filters=64, kernel_size=(1,8), strides=(1,2))
x_1 = cba(x_1, filters=64, kernel_size=(8,1), strides=(2,1))

x_2 = cba(inputs, filters=32, kernel_size=(1,16), strides=(1,2))
x_2 = cba(x_2, filters=32, kernel_size=(16,1), strides=(2,1))
x_2 = cba(x_2, filters=64, kernel_size=(1,16), strides=(1,2))
x_2 = cba(x_2, filters=64, kernel_size=(16,1), strides=(2,1))

x_3 = cba(inputs, filters=32, kernel_size=(1,32), strides=(1,2))
x_3 = cba(x_3, filters=32, kernel_size=(32,1), strides=(2,1))
x_3 = cba(x_3, filters=64, kernel_size=(1,32), strides=(1,2))
x_3 = cba(x_3, filters=64, kernel_size=(32,1), strides=(2,1))

x_4 = cba(inputs, filters=32, kernel_size=(1,64), strides=(1,2))
x_4 = cba(x_4, filters=32, kernel_size=(64,1), strides=(2,1))
x_4 = cba(x_4, filters=64, kernel_size=(1,64), strides=(1,2))
x_4 = cba(x_4, filters=64, kernel_size=(64,1), strides=(2,1))

x = Add()([x_1, x_2, x_3, x_4])

x = cba(x, filters=128, kernel_size=(1,16), strides=(1,2))
x = cba(x, filters=128, kernel_size=(16,1), strides=(2,1))

x = GlobalAveragePooling2D()(x)
x = Dense(classes)(x)
x = Activation("softmax")(x)

model = Model(inputs, x)

# initiate Adam optimizer
opt = keras.optimizers.adam(lr=0.00001, decay=1e-6, amsgrad=True)

# Let's train the model using Adam with amsgrad
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 1723, 1) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 862, 32) 288         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 128, 862, 32) 544         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 128, 862, 32) 1056        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_13 

In [0]:
# initiate Adam optimizer
# opt = keras.optimizers.adam(lr=0.00001, decay=1e-6, amsgrad=True)

# Let's train the model using Adam with amsgrad
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

# between class data generator
class MixupGenerator():
    def __init__(self, x_train, y_train, batch_size=16, alpha=0.2, shuffle=True):
        self.x_train = x_train
        self.y_train = y_train
        self.batch_size = batch_size
        self.alpha = alpha
        self.shuffle = shuffle
        self.sample_num = len(x_train)

    def __call__(self):
        while True:
            indexes = self.__get_exploration_order()
            itr_num = int(len(indexes) // (self.batch_size * 2))

            for i in range(itr_num):
                batch_ids = indexes[i * self.batch_size * 2:(i + 1) * self.batch_size * 2]
                x, y = self.__data_generation(batch_ids)

                yield x, y

    def __get_exploration_order(self):
        indexes = np.arange(self.sample_num)

        if self.shuffle:
            np.random.shuffle(indexes)

        return indexes

    def __data_generation(self, batch_ids):
        _, h, w, c = self.x_train.shape
        _, class_num = self.y_train.shape
        x1 = self.x_train[batch_ids[:self.batch_size]]
        x2 = self.x_train[batch_ids[self.batch_size:]]
        y1 = self.y_train[batch_ids[:self.batch_size]]
        y2 = self.y_train[batch_ids[self.batch_size:]]
        l = np.random.beta(self.alpha, self.alpha, self.batch_size)
        x_l = l.reshape(self.batch_size, 1, 1, 1)
        y_l = l.reshape(self.batch_size, 1)

        x = x1 * x_l + x2 * (1 - x_l)
        y = y1 * y_l + y2 * (1 - y_l)

        return x, y

In [0]:
# directory for model checkpoints
model_dir = "drive/My Drive/models"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

# early stopping and model checkpoint# early  
es_cb = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
chkpt = os.path.join(model_dir, 'speaker_sex2_.{epoch:02d}_{val_loss:.4f}_{val_acc:.4f}.hdf5')
cp_cb = ModelCheckpoint(filepath = chkpt, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

In [14]:
# train model
batch_size = 128
epochs = 10

from keras.models import load_model
model = load_model("drive/My Drive/models/speaker_sex2_.27_0.1955_0.9347.hdf5")

training_generator = MixupGenerator(x_train, y_train)()
model.fit_generator(generator=training_generator,
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    validation_data=(x_test, y_test),
                    epochs=epochs, 
                    verbose=1,
                    shuffle=True,
                    callbacks=[es_cb, cp_cb])

Epoch 1/10
27/27 [==============================] - 145s 5s/step - loss: 0.2516 - acc: 0.9444 - val_loss: 0.3257 - val_acc: 0.8585

Epoch 00001: val_loss did not improve from 0.19547
Epoch 2/10
27/27 [==============================] - 140s 5s/step - loss: 0.2912 - acc: 0.9398 - val_loss: 0.1938 - val_acc: 0.9339

Epoch 00002: val_loss improved from 0.19547 to 0.19379, saving model to drive/My Drive/models/speaker_sex2_.02_0.1938_0.9339.hdf5
Epoch 3/10
27/27 [==============================] - 140s 5s/step - loss: 0.3163 - acc: 0.8819 - val_loss: 0.7162 - val_acc: 0.6627

Epoch 00003: val_loss did not improve from 0.19379
Epoch 4/10
27/27 [==============================] - 140s 5s/step - loss: 0.3438 - acc: 0.8866 - val_loss: 0.2815 - val_acc: 0.8847

Epoch 00004: val_loss did not improve from 0.19379
Epoch 5/10
27/27 [==============================] - 140s 5s/step - loss: 0.3019 - acc: 0.9213 - val_loss: 0.2575 - val_acc: 0.8941

Epoch 00005: val_loss did not improve from 0.19379
Epoch 